# 5.4 단순한 계층 구현하기  

사과 쇼핑을 파이썬으로 구현해 보자  

    - MulLayer: 곱셈 노드
    - AddLayer: 덧셈 노드  

## 5.4.1 곱셈 계층  

    - forward(): 순전파  
    - backward(): 역전파  

곱셈 계층을 구현해 보자  

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

__init__()에서 x, y를 초기화, 두 변수는 순전파 시의 입력 값을 유지  
forward에서는 x, y를 인수로 받고 두값을 곱해 반환  
backward()에서는 상류에서 넘어온 미분(dout)에 순전파의 값을 서로 바꿔 곱한 뒤 하류로 흘림 (return)  

이를 이용해서 사과 쇼핑의 순전파를 다음과 같이 구현할 수 있다.  

In [3]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

각 변수에 대한 미분 값은 backward()로 구할 수 있다.  

In [4]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


호출 순서는 순전파와 반대이며 **backward가 받는 인수는 순전파 출력에 대한 미분 (dprice)이다.**  

## 5.4.2 덧셈 계층  

덧셈 계층은 다음과 같이 구현할 수 있다.  

In [5]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

덧셈 계층은 초기화가 필요 없어 __init__()에서는 아무일도 하지 않는다.  

이를 이용해서 다음을 구현해 보자  

<img src = "./image/fig_5_17.png" width = "40%">

In [6]:
# coding: utf-8

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650
